In [2]:
import metaworld
import random
from collections import defaultdict
from network import BNN
from planner import Planner

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

/home/dema/PycharmProjects/VBLRL/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
ml10 = metaworld.ML10()

/home/dema/PycharmProjects/VBLRL/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
class BayesianLayer(nn.Module):
    # cit https://github.com/Harry24k/bayesian-neural-network-pytorch
    def __init__(self, input_size, output_size, deterministic=False):
        
        super(BayesianLayer).__init__()
        
        # CLASS PARAMETERS
        # ---------------------------
        self.input_size = input_size
        self.output_size = output_size
        self.W_shape = (input_size, output_size)
        self.b_shape = (output_size)
        self.deterministic = deterministic
        
        # PARAMETERS (W, b)
        #-----------------------------
        # mean
        self.W_mu = torch.normal(torch.zeros(self.W_shape), torch.ones(self.W_shape))
        self.b_mu = torch.normal(torch.zeros(self.b_shape), torch.ones(self.b_shape))
        # variance 
        # ps: sono partito dalla varianza per ottenere il sigma, 
        # il codice originale fa il contrario (usando np.exp, log etc)
        self.W_sigma = torch.full(self.W_shape, 0.5)
        self.b_sigma = torch.full(self.b_shape, 0.5)
        
        # log(variance)   | rho in original codeBase
        self.W_log_sigma = torch.log(self.W_sigma)
        self.b_log_sigma = torch.log(self.b_sigma)
        
        # variational inference ---> compute update respect previous params
        self.W_mu_old = torch.Tensor(self.W_shape).detach()
        self.b_mu_old = torch.Tensor(self.b_shape).detach()
        self.W_sigma_old = torch.Tensor(self.W_shape).detach()
        self.b_sigma_old = torch.Tensor(self.b_shape).detach()
        self.W_log_sigma_old = torch.Tensor(self.W_shape).detach()
        self.b_log_sigma_old = torch.Tensor(self.b_shape).detach()
        
        # copy value of previous parameter
        self.save_old_parameters()
        
        
    def save_old_parameters(self): 
        self.W_mu_old = self.W_mu.clone()
        self.W_log_sigma_old = self.W_log_sigma.clone()
        self.b_mu_old = self.b_mu.clone()
        self.b_log_sigma_old = self.b_log_sigma.clone()
        
    def forward(self, X): 
        # link github method
        
        #sample W and b, return linear
        #weight = self.W_mu + torch.exp(self.W_log_sigma)*torch.randn_like(self.W_log_sigma)
        #bias = self.b_mu + torch.exp(self.b_log_sigma)*torch.randn_like(self.b_log_sigma)
        #return F.linear(X, weight, bias)
        
        gamma = X @self.W_mu + self.b_mu
        delta = X.pow(2)@ self.W_sigma + self.b_sigma
        # implement output as normal distrib result ???
        # " local reparametrization tricks"
    
    # this 2 method looks like set the random sample to a fixed value--> no perturbation
    def freeze(): pass
    def unfreeze(): pass
    def extra_repr():  pass #just return parameters
    
    def parameter_number(): pass
    # the 2 next as the same doc-string
    def get_parameters(): pass
    def get_old_parameters(): pass
    def set_parameters(): pass
    def save_old_parameters(): pass
    
class BNN(nn.Module): 
    # input = observation + action
    # output mean(reward, next_state), var(reward, next_state)
    
    def __init__(self, obs_shape, action_shape): pass
    def get_parameters(self) -> dict: pass
    def load_parameters(self, parameters): pass
    def forward(self, x): pass  # x = obs+act 
    
    

In [5]:
import scipy.stats as stats
import numpy as np

class CEM: 
    def __init__(self, 
                 cost_function, 
                 solution_dim, max_iters, 
                 pop_size=2, elite_size=1, 
                 upper_bound=None, lower_bound=None, 
                 epsilon=0.001, alpha=0.25
                ): 
        
        self.solution_dim = solution_dim
        self.pop_size = pop_size
        self.elite_size = elite_size
        
        self.cost_function = cost_function
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound

        self.max_iters = max_iters
        self.epsilon = epsilon
        self.alpha = alpha
        
        assert(elite_size < popsize)
        
        
    def solve(self, init_mean, init_var): 
        '''optimize the cost funct'''
        mean, var, time = init_mean, init_var, 0
        # pr distrib, truncated normal
        distrib = stats.truncnorm(-2, 2, loc=np.zeros_like(mean), scale=np.ones_like(var))
    
        while (t<self.max_iters) and np.max(var) > self.epsilon: 
            
            lower_bound_distrib = mean - self.lower_bound
            upper_bound_distrib = self.upper_bound - mean 
            constrained_var = np.minimum(
                np.minimum(np.square(lower_bound_distrib/2), np.square(upper_bound_distrib/2))
                var
            )
            
            samples = distrib.rvs(size=[self.pop_size, self.solution_dim]) * np.sqrt(constrained_var) + mean
            samples = samples.astype(np.float32)
            costs = self.cost_function(samples)
            elite = samples[np.argsort(-costs)][:self.elite_size]

            new_mean = np.mean(elite, axis=0)
            new_var  np.var(elite, axis=0)
            
            # just for fun, what if we set a decay in the alpha?
            # consider in the end more important the previous solution ??
            mean = self.alpha * mean + (1 - self.alpha)*new_mean
            var = self.alpha * var + (1 - self.alpha) * new_var
            
            time += 1
            
        return mean 
    
class Planner:
    
    def __init__(self, world_distrib_model, task_nets): 
        
        # params
        self.action_dim = action_dim
        self.plan_horizion = plan_horizon
        self.current_env_id = None
        self.system_current_state = None
        # ------------------------------
        # nets and optimizer
        self.world = world_distrib_model
        self.task_nets = task_nets
        self.optimizer = CEM(
            cost_function=self.cost_function 
            solution_dim =self.plan_horizon * self.action_dim, 
            max_iters=5, 
            pop_size=200, elite_size=50, 
            # shape upper/lower bound : solution dim = act*horizon (1D)
            upper_bound=np.tile(np.ones(self.action_dim), [self.plan_horizon])
            lower_bound=np.tile(-np.ones(self.action_dim), [self.plan_horizon]), 
            alpha=0.1)
        # ------------------------------
        
        # solution init:
        self.action_buffer = np.array([]).reshape(0, self.action_dim)
        self.prev_solution = np.tile(np.zeros(self.action_dim), [self.plan_horizion])
        self.init_var = np.tile(np.ones(self.action_dim)*1.5, [self.plan_horizion])
        # ------------------------------
    
    
    # "action" part
    def get_action(self, obs, env_index): # call to self.optimizer.solve
        self.current_env_id = env_index
        
        #Planning part : if not planning return random act (with np.random.uniform(-1, 1, solution_dim))
        # aggiungere dopo , casomai
        
        # se ho già generato una sequenza d'azione, ritorno la prima di queste
        if self.action_buffer.shape[0] > 0: 
            action, self.action_buffer = self.action_buffer[0], self.action_buffer[1:]
            return action
    
        # generate plan: 
        self.system_current_state = obs
        solution = self.optimizer.solve(mean=self.prev_sol, 
                                        var=self.init_var)
        
        self.prev_solution = np.concatenate(
            [np.copy(solution)[self.action_dim:], np.zeros(self.action_dim)]
        )
        self.action_buffer = solution[:self.action_dim](-1, self.action_dim)
        return self.get_action(obs, env_idx)
    
    def get_weight_as_vect(self) -> np.array: pass
    def restore_weight_from_vect(self): pass
    
    
    
    # equivalent to cost funct
    def rollout(self, env, action_sequence):
        # not work, should keep a env id inside?
        assert(env==self.current_env_id)
        # num op --> shape action sequence[0]
        action_seq.view(-1, self.plan_horizion, self.action_dim)
        
        # current observation 
        current_obs = ...
        # prepare rewards = zeros
        for step in range(self.plan_horizion): 
            
            current_action = action_sequence[t]
            
            dynamic_output = self.task_nets[self.current_env_id].infer(current_observation, current_action)
            reward, reward_mu, reward_var, new_state, new_state_mu, new_state_var = dynamic_output
            
            # sum up reward for each state
            # in this part there is a lot of stuff about reshape, transpose etc
            
        return -rewards
            
            
            

In [18]:
class ReplayBuffer:
    # incomplete as funct list ---> reset, use index etc
    # ragno usa namedTuple, try to understand why
    def __init__(self, memory_size=50000): pass
    def sample_batch(self, batch_size=32): pass
    def append(self, s, a, r, d, new_s): pass
    def remaining_space(self): pass
    def used_space(self): pass

class MultiTargetReplayBuffer: 
    #set of replay buffer over same env, different config of task
    def __init__(self, memory_size_for_buffer, env, task): pass
    def append(self, task, s, a, r, d, new_s): pass
    def random_batch(self, task): pass
    def load_buffer_from_path(self, task, path): pass
    # manca clear_buffer

In [ ]:
class VBLRL_alg: 
    
    def __init__(self, agent, world_model, task_specific): pass
    def init_task_specific_from_world(task_id): pass
    def update_buffer(): pass 
    def update_world_model( buffers_data ): pass
    def update_task_specific( buffer_data): pass
    
    def run(): 
        pass
        for task_id in range(len(tasks)): 
            init_task_specific_from_world(task_id)
            
            
            # NON SO UN CAZZO 
            
            for i in range(iteration): 
                for t in range(max_time_horizion): 
                    agent.get_action() # how set parameters
                    
    def train(self): 
        for env_id in range(self.env_set)

# LAUNCHER ml10

In [9]:
def prepare_env(): 
    """create 10 env with random task (random as init condition)"""
    envs_name = list(ml10.train_classes.keys())
    tasks_for_env = defaultdict(list)
    
    for task in ml10.train_tasks: 
        tasks_for_env[task.env_name].append(task)
    
    envs = []
    for env_name in envs_name: 
        env = ml10.train_classes[env_name]()
        env.set_task(tasks_for_env[env_name][random.randint(0, 49)])
        envs.append(env)
    
    return envs

def get_NN_for_exp(num_task, obs_shape, action_shape): 
    world_distrib = BNN(obs_shape, action_shape)
    task_specific_BNNs = [BNN(obs_shape, action_shape) for _ in range(num_task)]
    return world_distrib, task_specific_BNNs

In [10]:
envs = prepare_env()
observation_dim = envs[0].observation_space.shape[0]
action_dim = envs[0].action_space.shape[0]

/home/dema/PycharmProjects/VBLRL/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [16]:
world_distrib, task_specific_BNN = get_NN_for_exp(len(envs), observation_dim, action_dim)
planner = Planner(world_distrib, task_specific_BNN)